## Imports

In [ ]:
import random
%matplotlib inline
import cv2
import torch
import numpy as np
from PIL import Image
from skimage.transform import resize
from matplotlib import pyplot as plt
from torchvision.transforms import ToTensor
from torch.optim import RMSprop
from skimage import data, io, filters
from scipy.stats.kde import gaussian_kde
import matplotlib.pyplot as plt

In [ ]:
def mrf_prior(x):
    return x**2

def denoise(noisy_img):
    M, N = noisy_img.shape
    y = noisy_img.copy()
    maxiter = 5*M*N

    for iter in range(maxiter):
        i = np.random.randint(M)
        j = np.random.randint(N)
        neighbours = get_neighbours(i, j, M, N)

        enrg_1 = enrg(1, y[i, j], y, neighbours)
        enrg_0 = enrg(0, y[i, j], y, neighbours)

        if enrg_1 > enrg_0:
            y[i, j] = 1
        else:
            y[i, j] = 0
        
        if iter % 100000 == 0:
            print (f'Completed {iter} iterations out of {maxiter}. Denoized pixels are: {diff(y, noisy_img)}%')

    return y

def diff(y, y_old):
    diff = abs(y - y_old) / 2
    return (100.0 * np.sum(diff)) / np.size(y)

def enrg(new, old, y, neighbours):
    lmda = -100
    return (new - old)**2 + lmda * np.sum((new - y[neighbour[0], neighbour[1]])**2 for neighbour in neighbours)

def get_neighbours(i, j, M, N):
    neighbours = []
    if i > 0:
        neighbours.append([i-1, j])
    if i < M-1:
        neighbours.append([i+1, j])
    if j > 0:
        neighbours.append([i, j-1])
    if j < N-1:
        neighbours.append([i, j+1])

    return neighbours

def mrf_potential(X, noisy, a):
    
    energy_1 = ((noisy - X)**2).sum()
    
    energy_2 = 0
    energy_2 += mrf_prior(X[:, 1: ] - X[:, :-1]).sum()
    energy_2 += mrf_prior(X[:-1, :] - X[ 1:, :]).sum()
    energy_2 += mrf_prior(X[:-1,:-1] - X[1:, 1:]).sum()
    energy_2 += mrf_prior(X[1:, :-1] - X[ :-1, 1:]).sum()
    
    return energy_1 + a*energy_2*2

def add_noise(orig_img, thresh):

    N, M = orig_img.shape
    noisy_img = orig_img.copy()
    Gaussian_noise = np.random.rand(N, M)
    Gaussian_noise[Gaussian_noise < 1-thresh] = 0
    Gaussian_noise[Gaussian_noise >= 1-thresh] = 1

    noisy_img = (noisy_img + Gaussian_noise) % 2
    
    return noisy_img

def binarize_img(im):
    bin_im = im / 255.
    bin_im.flags.writeable = True
    bin_im[bin_im < 0.5] = 0
    bin_im[bin_im >= 0.5] = 1
    return bin_im

## Part A

### Converting the image to binary

In [ ]:
image = data.camera()
print("Dimensions of the image are:", image.shape)
plt.imshow(image, cmap='gray')

In [ ]:
gt = image
image = image/255

In [ ]:
x, y = image.shape
print("Nd array of image cameraman : \n",image)
mean = 0
var = 0.01
sigma = np.sqrt(var)
Gaussian_noise = np.random.normal(loc=mean, scale=sigma, size=(x,y))
print("Gaussian Noise Nd - array : \n", Gaussian_noise)

# Adding Gaussian Noise


In [ ]:
g = image + Gaussian_noise
noisy = g
print("Noisy Gaussian Array : \n")
print(noisy)
g = g*255
plt.imshow(g, cmap='gray')

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(131)
plt.title('Gaussian Noisy Image')
plt.imshow(g, cmap='gray')
plt.subplot(132)
plt.title('Actual Image')
plt.imshow(gt, cmap='gray')
plt.show()

In [ ]:
bin_image = binarize_img(gt)
plt.imshow(bin_image, cmap='gray')

# We try to introduce Some sample of noisy Image's in which we introduce Some percentage of noise on the basis of 0.05 threshold value.

In [ ]:
fig = plt.figure(figsize=(12, 6))

for i in range(0, 5 + 1):
    thresh = i/20
    noisy_img = add_noise(bin_image, thresh)
    plt.subplot(2, 3, i+1)
    plt.title(f'Noise: {thresh*100}%')
    plt.axis('off')
    plt.imshow(noisy_img, cmap='gray')
plt.show()

## Testing

In [ ]:
fig = plt.figure(figsize=(8, 20))

for i in range(1, 5 + 1):
    thresh = i/20
    print(f"Denoising for noise level: {thresh*100}")

    noisy_img = add_noise(bin_image, thresh)
    denoised_img = denoise(noisy_img)

    plt.subplot(5, 2, 2*i-1)
    plt.title(f'Noise: {thresh*100}%')
    plt.axis('off')
    plt.imshow(noisy_img, cmap='gray')

    plt.subplot(5, 2, 2*i)
    plt.title(f'Denoised image ({diff(noisy_img, denoised_img)}%)')
    plt.axis('off')
    plt.imshow(denoised_img, cmap='gray')

    print()
plt.show()

# Relative Root Mean Squared Error Analysis  

In [ ]:
tensor_convert = ToTensor()
noisy = tensor_convert(noisy)[0]
gt    = tensor_convert(gt)[0]

In [ ]:
to_tensor = ToTensor()
RRMSE = lambda x: (((gt - x)**2).sum() / (gt**2).sum())**0.5
X = torch.zeros_like(noisy)
X.requires_grad = True
errors = []
losses = []
images = []
alpha  = 0.15
optimizer = RMSprop([X])
n_it   = 100

In [ ]:
for it in range(n_it):
    optimizer.zero_grad()
    loss = mrf_potential(X, noisy, alpha)
    loss.backward()
    optimizer.step()
    errors.append(RRMSE(X).cpu().detach())
    losses.append(loss.item())
    images.append(np.array(X.cpu().detach()).astype(np.uint8))

In [ ]:
print("RRMSE Initial: {}".format(RRMSE(noisy)))
print("RRMSE Final  : {}".format(errors[-1]))

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.title('Objective Function')
plt.plot(losses)
plt.subplot(122)
plt.title('RRMSE')
plt.plot(errors)
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(131)
plt.title('Noisy Image')
plt.imshow(noisy.cpu(), cmap='gray')
plt.subplot(132)
plt.title('Denoised Image')
plt.imshow(X.detach().cpu(), cmap='gray')
plt.subplot(133)
plt.title('Ground Truth')
plt.imshow(gt.cpu(), cmap='gray')
plt.show()

Conclusion:
1. Initially, when we denoise our noisy image we will train our epochs and for every epoch, we get some objective function loss at every epoch.
2. We will be getting a denoised image after energy minimization. and try to compare with ground truth which is our real image we can clearly see some of the noise will be removed by minimization of energy with conditional probability of neighbor node through normalization of the product of minimal click which is our subset of a random variable taken from X.
